In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model

import matplotlib.pyplot as plt

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Google Collab
# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/user_ratings.csv')
# df = df[['Username','BGGId','Rating']]
# game_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/games.csv')

In [17]:
## Local pc
df = pd.read_csv('C:/Users/John/Documents/LHL Lecture Material/boardgame-recommender/data/user_ratings.csv')
df = df[['Username','BGGId','Rating']]
game_df = pd.read_csv('C:/Users/John/Documents/LHL Lecture Material/boardgame-recommender/data/games.csv')

In [18]:
# Ensure all datapoints are the same type
df['Username'] = df['Username'].astype(str)
df['BGGId'] = df['BGGId'].astype(int)
df['Rating'] = df['Rating'].astype('float32')

In [19]:
def recent_game(date):
    """ 
    Transform the PublishedYear Column to a binary classification. If it was before 2016, it is old(0), otherwise new(1)
    
    Parameters:
        date(int): the date publication    

    Returns:
        0 or 1 depending on the date provided. 
    """
    if date > 2016:
        return 1
    else: 
        return 0
    
game_df['new_or_old'] = game_df['YearPublished'].apply(recent_game)

In [20]:
# Create new dataframe with all the content based features
feats = ['BGGId','new_or_old','BayesAvgRating','BestPlayers','MfgPlaytime','NumUserRatings','NumComments']
feats_df = game_df[feats]
comb_feats_df = df.merge(feats_df, on='BGGId')
feats = ['new_or_old','BayesAvgRating','BestPlayers','MfgPlaytime','NumUserRatings','NumComments']

In [21]:
# Encode the BGGId and Username to a certain value via a dictionary
user_ids = df['Username'].unique().tolist()
game_ids = df['BGGId'].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
game2game_encoded = {x: i for i,x in enumerate(game_ids)}

In [22]:
# Create new columns 
comb_feats_df['user'] = comb_feats_df['Username'].map(user2user_encoded)
comb_feats_df['game'] = comb_feats_df['BGGId'].map(game2game_encoded)

In [23]:
# Create train and test sets that are scaled
X = comb_feats_df[['user','game','new_or_old','BayesAvgRating','BestPlayers','MfgPlaytime','NumUserRatings','NumComments']]
y = comb_feats_df[['Rating']]
scaler = MinMaxScaler()
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)
x_train[feats], x_test[feats] = scaler.fit_transform(x_train[feats]), scaler.transform(x_test[feats])
y_train, y_test = scaler.fit_transform(y_train), scaler.transform(y_test)

In [11]:
# Hybrid Model
embed_size = 50
num_users = len(user2user_encoded)
num_games = len(game2game_encoded)
num_feats = len(feats)

#input layer
user_in = Input(name='user_in', shape=(1,))
prod_in = Input(name='prod_in', shape=(1,))

#matrix factorization
mf_user_emb = Embedding(name='mf_user_emb', input_dim=num_users, output_dim = embed_size)(user_in)
mf_user_flat = Flatten(name='mf_user_flat')(mf_user_emb)
mf_prod_emb = Embedding(name='mf_prod_emb', input_dim=num_games, output_dim=embed_size)(prod_in)
mf_prod_flat = Flatten(name='mf_prod_flat')(mf_prod_emb)
mf_dot = Dot(name='mf_dot', normalize=True, axes=1)([mf_user_flat, mf_prod_flat])

#neural network
nn_user_emb = Embedding(name='nn_user_emb', input_dim = num_users, output_dim = embed_size)(user_in)
nn_user_flat = Flatten(name='nn_user_flat')(nn_user_emb)
nn_prod_emb = Embedding(name='nn_prod_emb', input_dim=num_games, output_dim= embed_size)(prod_in)
nn_prod_flat = Flatten(name='nn_prod_flat')(nn_prod_emb)
nn_concat = Concatenate()([nn_user_flat, nn_prod_flat])
nn_dense = Dense(name='nn_dense', units=int(embed_size/2), activation='relu')(nn_concat)

# Product features
feats_in = Input(name='feat_in', shape=(num_feats,))
feats_dense = Dense(name='feat_dense', units = num_feats, activation = 'relu')(feats_in)

#Merge both together
combo = Concatenate()([mf_dot, nn_dense, feats_dense])
y_out = Dense(name='y_out', units=1, activation='linear')(combo)

hyb_model = Model(inputs=[user_in, prod_in, feats_in], outputs = y_out, name='hybrid_model')
hyb_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
hyb_model.summary()

Model: "hybrid_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_in (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 prod_in (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 nn_user_emb (Embedding)        (None, 1, 50)        20568750    ['user_in[0][0]']                
                                                                                                  
 nn_prod_emb (Embedding)        (None, 1, 50)        1096250     ['prod_in[0][0]']                
                                                                                       

In [12]:
history = hyb_model.fit(x=[x_train['user'], x_train['game'], x_train[feats]], y=y_train, epochs=15, batch_size = 5000, verbose=True)

Epoch 1/15
3031/3031 [==============================] - 119s 37ms/step - loss: 0.0227 - mean_squared_error: 0.0227
Epoch 2/15
3031/3031 [==============================] - 55s 18ms/step - loss: 0.0136 - mean_squared_error: 0.0136
Epoch 3/15
3031/3031 [==============================] - 52s 17ms/step - loss: 0.0114 - mean_squared_error: 0.0114
Epoch 4/15
3031/3031 [==============================] - 52s 17ms/step - loss: 0.0088 - mean_squared_error: 0.0088
Epoch 5/15
3031/3031 [==============================] - 50s 16ms/step - loss: 0.0078 - mean_squared_error: 0.0078
Epoch 6/15
3031/3031 [==============================] - 49s 16ms/step - loss: 0.0071 - mean_squared_error: 0.0071
Epoch 7/15
3031/3031 [==============================] - 48s 16ms/step - loss: 0.0067 - mean_squared_error: 0.0067
Epoch 8/15
3031/3031 [==============================] - 46s 15ms/step - loss: 0.0063 - mean_squared_error: 0.0063
Epoch 9/15
3031/3031 [==============================] - 45s 15ms/step - loss: 0.0060 - 

In [14]:
## Google colab
# hyb_model.save('/content/drive/MyDrive/Colab Notebooks/hybrid_model')

In [ ]:
## Local PC
hyb_model.save('C:/Users/John/Documents/LHL Lecture Material/boardgame-recommender/src/models/hybrid_model')

In [15]:
y_pred = hyb_model.predict(x=[x_test['user'], x_test['game'],x_test[feats]])

118389/118389 [==============================] - 230s 2ms/step


In [16]:
mean_squared_error(y_test, y_pred, squared=False)

0.13115211

In [25]:
saved_model = tf.keras.models.load_model('C:/Users/John/Documents/LHL Lecture Material/boardgame-recommender/src/models/hybrid_model')

In [93]:
def user_recommendations(user, n_games,model):
    """
        Takes in a user, n_games and a model to returns their top n_games recommendations that they have not rated yet.

        Parameters:
            user(str): name of user
            n_games(int): number of games to show
            model: the model to use for predictions
            
        Returns:
            A list with each game as an index with a column for their estimated rating
    """
    # Remove the boardgames that the user have rated
    boardgame_id = comb_feats_df['BGGId'].unique()
    user_list = comb_feats_df.loc[comb_feats_df['Username']==user, 'BGGId']
    user_list = np.setdiff1d(boardgame_id, user_list)

    # Create dataframe with username, bggid and features
    testset = [[user2user_encoded[user], game2game_encoded[bggid]] for bggid in user_list]
    testset = pd.DataFrame(testset, columns=['user_encode','bggid_encode'])
    feats = ['BGGId','new_or_old','BayesAvgRating','BestPlayers','MfgPlaytime','NumUserRatings','NumComments']
    feat_df = game_df[game_df['BGGId'].isin(user_list)][feats]
    feat_df['bggid_encode'] = feat_df['BGGId'].map(game2game_encoded)
    feat_df = feat_df.drop('BGGId',axis=1)
    testset = pd.merge(testset, feat_df, on='bggid_encode')
    feats = ['new_or_old','BayesAvgRating','BestPlayers','MfgPlaytime','NumUserRatings','NumComments']

    # Test model on the testset and determine the boardgames with the highest rating
    predictions = model.predict(x=[testset['user_encode'],testset['bggid_encode'],testset[feats]])
    # Reshaped the predictions array by Transposing it. 
    top_ratings_idx = predictions.T[0].argsort()[::-1][:n_games]
    bgg_ids = user_list[top_ratings_idx]
    bgg_name = [game_df.loc[game_df['BGGId'] ==id]['Name'].values[0] for id in bgg_ids]
    print(f'Top boardgames for {user} in order are: \n {bgg_name}')

In [94]:
user_recommendations('bennygui', 5, saved_model)

673/673 [==============================] - 3s 4ms/step
Top boardgames for bennygui in order are: 
 ['The Campaign for North Africa: The Desert War 1940-43', 'Agricola', 'Case Blue', 'Power Grid', 'Gloomhaven']
